<a href="https://colab.research.google.com/github/Gabriela-Alcaide/Aprendizado_por_Reforco_IC/blob/main/Bike_Workers_%2B_GUROBI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classe do Ambiente

In [80]:
import random
from itertools import product, chain
import numpy as np

class BikeWorkers_Environment:
    def __init__ (self, n_W, n_S):
        self.n_W = n_W
        self.n_S = n_S

        self.state = tuple([0]*(self.n_W+1))

        self.states = list(product(range(n_S+1), repeat=n_W+1))
        self.actions = tuple(x for x in range(n_W+1))

        self.Pcw = 0.8
        self.Pcx = np.random.rand(n_S)
        self.Pcx = (1-self.Pcw)*self.Pcx/self.Pcx.sum()

        self.Pwxy = np.random.rand(n_W,n_S)

        self.Gx = .5 + np.random.rand(n_S)
        self.Hxy = np.random.rand(n_W,n_S)


    def reset(self):
        self.state = tuple([0]*(self.n_W+1))

    def nextState(self,state, action):
        nextStates = {}

        # Lista de possíveis valores para o próximo estado.
        cart_dim = []
        # Probabilidade de cada possível valor.
        cart_dim_prob = []

        # Lista de todos os serviços.
        Cs = list(range(self.n_S+1))
        # Concatena as probabilidades.
        PCs = np.concatenate(([self.Pcw], self.Pcx))

        # Atualiza o conjunto de possibilidades.
        cart_dim = cart_dim + [Cs]
        cart_dim_prob = cart_dim_prob + [PCs]

        # Para cada trabalhadora.
        for w in range(self.n_W):
          # Se está "waiting":
          if state[w+1] == 0:  # Tudo é determinista.
            PWs = [1]
            # Se a ação foi alocar um serviço para essa entregadora, ela o recebe.
            if action == w+1:
              Ws = [state[0]]   # Se a trabalhadora foi escolhida, aloca o serviço atual
            # Se não, continua "waiting".
            else:
              Ws = [0]
          # Se já estava ocupada.
          else:
            # Pode entregar o serviço e ir para "waiting" ou continuar trabalhando no serviço atual.
            Ws = [0, state[w+1]]
            # A probabilidade de entregar o serviço e a de não entregá-lo (continuar nele).
            PWs = [self.Pwxy[w][state[w+1]-1] , 1 - self.Pwxy[w][state[w+1]-1] ]

          # Atualiza.
          cart_dim = cart_dim + [Ws]
          cart_dim_prob = cart_dim_prob + [PWs]

        # Todos os estados futuros possíveis, e suas probabilidades.
        states = list(product(*cart_dim))
        probs = list(product(*cart_dim_prob))

        # Coloca no dicionário.
        for s, p in zip(states, probs):
            nextStates[s] = np.prod(p)

        return nextStates

    def previousStates(self,state, action):
      previousStates = {}
      for s in self.states:
        # Estados seguintes (chaves) e suas probabilidades de transição (valores).
        ss = self.nextState(s,action)

        # Para cada estado seguinte.
        for s_ in ss:
          if s_ == state:
            # Probabilidade de transição de s para s_ (state).
            prob = ss[s_]
            previousStates[s] = prob
      return previousStates

    def reward_structure(self, state, action, next_state):
        # Recompensas e custos para cada trabalhadora em cada transição do Processo Markoviano.
        # Inicialmente, em cada transição, cada trabalhadora não ganhou nem gastou nada.
        ganhos = np.zeros(self.n_W)
        custos = np.zeros(self.n_W)

        # Para cada trabalhadora.
        for x in range(self.n_W):
          # Se está entregando um serviço.
          if state[x+1] > 0:
            # Tem um custo.
            custos[x] = self.Hxy[x][state[x+1]-1]
            # Se termina o serviço nessa transição, ganha por sua entrega.
            if next_state[x+1] == 0:
              ganhos[x] = self.Gx[state[x+1]-1]

        return ganhos, custos

    def reward(self,state, action, next_state):
        # Recompensas do sistema todo em cada transição.
        ganhos, custos = self.reward_structure(state, action, next_state)
        return np.sum(ganhos) - np.sum(custos)

    def feasible_actions(self):
        actions = [0]
        if self.state[0] > 0:
          for w in range(self.n_W):
            if self.state[w+1] == 0:
              actions = actions + [w+1]
        return actions

    def simulateStep(self,state,action):
        nextStates = self.nextState(state,action)
        nextState = random.choices( list( nextStates.keys() ), weights = list( nextStates.values() ), k=1 )[0]
        r = self.reward(state, action, nextState)

        return nextState, r

    def step(self,action):
        self.state, r  = self.simulateStep(self.state,action)
        return self.state, r

    def render(self):
        print(self.state)


# PROGRAMAÇÃO LINEAR - instanciando mdp

In [2]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 38.7 MB/s eta 0:00:00


In [9]:
import gurobipy as gp
from gurobipy import GRB

In [10]:
n_W = 1
n_S = 2
maxSteps = 100

mdp = BikeWorkers_Environment(n_W,n_S)

print('Ganho obtido com cada serviço')
print(mdp.Gx)

print('Custo médio de cada serviço para cada trabalhadora')
print(mdp.Hxy/(mdp.Pwxy))


Ganho obtido com cada serviço
[1.42668697 1.06373945]
Custo médio de cada serviço para cada trabalhadora
[[0.1869272 0.8509917]]


In [11]:
gamma = 0.8

# PROGRAMAÇÃO LINEAR PRIMAL

In [56]:
# Criando modelo.
model = gp.Model()

In [57]:
# Variáveis de decisão: função valor dos estados.
V = model.addVars(mdp.states, vtype=GRB.CONTINUOUS, name="V", lb=-GRB.INFINITY)

In [51]:
'''model.setObjective(
    gp.quicksum(
        V[j] for j in mdp.states
    ),
    GRB.MINIMIZE
)'''

In [58]:
model.setObjective(
    V[(0,0)],
    GRB.MINIMIZE
)

In [59]:
V

{(0, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 2): <gurobi.Var *Awaiting Model Update*>,
 (1, 0): <gurobi.Var *Awaiting Model Update*>,
 (1, 1): <gurobi.Var *Awaiting Model Update*>,
 (1, 2): <gurobi.Var *Awaiting Model Update*>,
 (2, 0): <gurobi.Var *Awaiting Model Update*>,
 (2, 1): <gurobi.Var *Awaiting Model Update*>,
 (2, 2): <gurobi.Var *Awaiting Model Update*>}

In [60]:
for s in mdp.states:
  for a in mdp.actions:
    r = 0
    Transitions = mdp.nextState(s,a)
    for ss in Transitions:
      r = r + Transitions[ss]*(mdp.reward(s,a,ss))

    model.addConstr(V[s] - gamma * gp.quicksum(Transitions[ss]*V[ss] for ss in Transitions) >= r, name=f"func_valor")

In [61]:
for c in model.getConstrs():
    print(f"{c.ConstrName}: {model.getRow(c)}")
    print(f"{c.ConstrName}: {c.RHS}")

In [62]:
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 18 rows, 9 columns and 92 nonzeros
Model fingerprint: 0x4568e9ea
Coefficient statistics:
  Matrix range     [1e-03, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 1e+00]
Presolve removed 16 rows and 7 columns
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.133818e+00   0.000000e+00      0s
       1    5.7330828e-01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.733082821e-01


In [63]:
for v in model.getVars():
    print(f"{v.VarName}: {v.X}")

V[0,0]: 0.5733082821474575
V[0,1]: 1.7933478991811749
V[0,2]: 0.705692829948757
V[1,0]: 1.4346783193449402
V[1,1]: 1.7933478991811749
V[1,2]: 0.7056928299487568
V[2,0]: 0.5733082821474575
V[2,1]: 1.7933478991811747
V[2,2]: 0.7056928299487569


# Programação Linear Dual

In [72]:
# Criando modelo.
model_dual = gp.Model()

In [73]:
# Variáveis de decisão: função valor dos estados.
x = {}
for s in mdp.states:
  x[s] = {}
  for a in mdp.actions:
    x[s][a] = model_dual.addVar(vtype=GRB.CONTINUOUS, name="x", lb=0)

In [74]:
x

{(0, 0): {0: <gurobi.Var *Awaiting Model Update*>,
  1: <gurobi.Var *Awaiting Model Update*>},
 (0, 1): {0: <gurobi.Var *Awaiting Model Update*>,
  1: <gurobi.Var *Awaiting Model Update*>},
 (0, 2): {0: <gurobi.Var *Awaiting Model Update*>,
  1: <gurobi.Var *Awaiting Model Update*>},
 (1, 0): {0: <gurobi.Var *Awaiting Model Update*>,
  1: <gurobi.Var *Awaiting Model Update*>},
 (1, 1): {0: <gurobi.Var *Awaiting Model Update*>,
  1: <gurobi.Var *Awaiting Model Update*>},
 (1, 2): {0: <gurobi.Var *Awaiting Model Update*>,
  1: <gurobi.Var *Awaiting Model Update*>},
 (2, 0): {0: <gurobi.Var *Awaiting Model Update*>,
  1: <gurobi.Var *Awaiting Model Update*>},
 (2, 1): {0: <gurobi.Var *Awaiting Model Update*>,
  1: <gurobi.Var *Awaiting Model Update*>},
 (2, 2): {0: <gurobi.Var *Awaiting Model Update*>,
  1: <gurobi.Var *Awaiting Model Update*>}}

In [75]:
# Função objetivo.
model_dual.setObjective(
    gp.quicksum(
        x[s][a] * sum(
            mdp.reward(s, a, ss) * p_transicao
            for ss, p_transicao in mdp.nextState(s, a).items()
        )
        for s in mdp.states
        for a in mdp.actions
    ),
    GRB.MAXIMIZE
)

In [76]:
# Restrição.
for s in mdp.states:
  if s == (0,0):
    model_dual.addConstr(gp.quicksum(x[s][a] - gamma * gp.quicksum(mdp.previousStates(s,a)[ss] * x[ss][a] for ss in mdp.previousStates(s,a)) for a in mdp.actions) == 1, name=f"restricao_{s}")
  else:
    model_dual.addConstr(gp.quicksum(x[s][a] - gamma * gp.quicksum(mdp.previousStates(s,a)[ss] * x[ss][a] for ss in mdp.previousStates(s,a)) for a in mdp.actions) == 0, name=f"restricao_{s}")

In [77]:
for c in model_dual.getConstrs():
    print(f"{c.ConstrName}: {model_dual.getRow(c)}")
    print(f"{c.ConstrName}: {c.RHS}")

In [78]:
model_dual.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 9 rows, 18 columns and 92 nonzeros
Model fingerprint: 0x636180dd
Coefficient statistics:
  Matrix range     [1e-03, 1e+00]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 8 columns
Presolve time: 0.00s
Presolved: 8 rows, 10 columns, 42 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.9321272e+30   3.971114e+30   3.932127e+00      0s
       7    5.7330828e-01   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.733082821e-01


In [79]:
x

{(0, 0): {0: <gurobi.Var x (value 0.0)>,
  1: <gurobi.Var x (value 3.8097117597918464)>},
 (0, 1): {0: <gurobi.Var x (value 0.3902882402081556)>,
  1: <gurobi.Var x (value 0.0)>},
 (0, 2): {0: <gurobi.Var x (value 0.0)>, 1: <gurobi.Var x (value 0.0)>},
 (1, 0): {0: <gurobi.Var x (value 0.0)>,
  1: <gurobi.Var x (value 0.584400051942508)>},
 (1, 1): {0: <gurobi.Var x (value 0.0)>,
  1: <gurobi.Var x (value 0.08117717664643773)>},
 (1, 2): {0: <gurobi.Var x (value 0.0)>, 1: <gurobi.Var x (value 0.0)>},
 (2, 0): {0: <gurobi.Var x (value 0.11802788800545351)>,
  1: <gurobi.Var x (value 0.0)>},
 (2, 1): {0: <gurobi.Var x (value 0.0)>,
  1: <gurobi.Var x (value 0.016394883405601135)>},
 (2, 2): {0: <gurobi.Var x (value 0.0)>, 1: <gurobi.Var x (value 0.0)>}}

#Execution 1: Manual Control

Expand the cell, before executing it.

In [ ]:
import time
import numpy as np
from IPython.display import clear_output

def Simulate(env,maxSteps):
  # Simula o ambiente até atingir um máximo de passos, passado via parâmetro.
  env.reset()
  tempo = .25
  clear_output(wait=True)
  env.render()

  steps = 0
  eval = 0

  while steps < maxSteps:
      # Para cada passo.

      # Obtém as ações possíveis.
      actions = env.feasible_actions()

      # Se há ações possíveis além do "refuse".
      if len(actions) > 1:
        # Pede para atribuir o serviço à uma entregadora.              ?
        command = input(f"{steps}({eval}) Choose a worker (" + str(env.feasible_actions()) + ") to move: ")
        for a in command:
          # Obtém o estado atual do ambiente.
          s = env.state

          # Entregadora que receberá o serviço.
          a = int(a)
          # Para cada ação possível nesse passo.
          if a in env.actions:
              # Dá o passo e o avalia.
              _ , r = env.step(a)
              eval = eval + r
              steps += 1

          clear_output(wait=True)
          env.render()
          time.sleep(tempo)
          break
      else:
        a = actions[0]

        _ , r = env.step(a)
        eval = eval + r

        steps += 1
        clear_output(wait=True)
        env.render()
        time.sleep(tempo)

  print(f"Congratulations. You earn {eval}.")


  return steps

In [ ]:
n_W = 3
n_S = 2
maxSteps = 30

env = BikeWorkers_Environment(n_W,n_S)
evalGrid = Simulate(env,maxSteps)




(2, 1, 0, 0)


KeyboardInterrupt: Interrupted by user

# Execution 2: Policies
Expand the cells, before executing them.

In [ ]:
import time
import numpy as np
from IPython.display import clear_output

def EvaluatePolicyMC(env,policy,maxSteps,nSamples):
  # Monte Carlo.

  memEval = []

  # Para cada teste da política.
  for i in range(nSamples):
    env.reset()
    steps = 0
    eval = 0

    # Há uma quantidade máxima de passos por teste. Se não, rodaria infinitamente.
    while steps < maxSteps:
      # Estado do ambiente.
      s = env.state
      # Ação que a política define que deve ser tomada.
      a = policy[s]
      # Toma a ação e receve a recompensa por ela.
      _, r = env.step(a)
      # Atualiza a recompensa.
      eval = eval + r
      steps += 1

    # Soma da recompensa de todas as simulações.
    memEval = memEval + [eval]

    clear_output(wait=True)
    print(f'Simulating {(i+1)/nSamples}')

    # Média das recompensas.
    print(f'The average gain in simulations was {np.mean(memEval)}.')

  return memEval

def SimulatePolicy(env,policy,maxSteps):
  # Simulação de uma política.
  env.reset()
  tempo = .5
  clear_output(wait=True)
  env.render()

  eval = 0
  steps = 0

  # Há quantidade limite de passos.
  while steps < maxSteps:
    # Estado atual.
    s = env.state
    # Ação que a política define que deve ser tomada.
    a = policy[s]

    # Toma a ação.
    _, r = env.step(a)
    # Atualiza a recompensa.
    eval = eval + r
    steps += 1

    clear_output(wait=True)
    print(f'Steps: {steps}')
    env.render()
    time.sleep(tempo)

  # Recompensa gerada pela política após todos os passos.
  print(f"Congratulations. You earn {eval}.")

  return eval



Cria o ambiente que será utilizado nas simulações.

In [ ]:
n_W = 3
n_S = 2
maxSteps = 100

env = BikeWorkers_Environment(n_W,n_S)

print('Ganho obtido com cada serviço')
print(env.Gx)

print('Custo médio de cada serviço para cada trabalhadora')
print(env.Hxy/(env.Pwxy))


Ganho obtido com cada serviço
[1.26358147 0.86106335]
Custo médio de cada serviço para cada trabalhadora
[[1.54027377 5.01511069]
 [0.29335982 0.84023723]
 [1.61116599 0.20374301]]


Define by trial and error a policy by setting the variable **policy**.


In [ ]:


# policy = {
#           (0,0,0,0): 0, (0,0,0,1): 0, (0,0,0,2): 0, (0,0,1,0): 0, (0,0,1,1): 0, (0,0,1,2): 0, (0,0,2,0): 0, (0,0,2,1): 0, (0,0,2,2): 0,
#           (0,1,0,0): 0, (0,1,0,1): 0, (0,1,0,2): 0, (0,1,1,0): 0, (0,1,1,1): 0, (0,1,1,2): 0, (0,1,2,0): 0, (0,1,2,1): 0, (0,1,2,2): 0,
#           (0,2,0,0): 0, (0,2,0,1): 0, (0,2,0,2): 0, (0,2,1,0): 0, (0,2,1,1): 0, (0,2,1,2): 0, (0,2,2,0): 0, (0,2,2,1): 0, (0,2,2,2): 0,
#           (1,0,0,0): 0, (1,0,0,1): 0, (1,0,0,2): 0, (1,0,1,0): 0, (1,0,1,1): 0, (1,0,1,2): 0, (1,0,2,0): 0, (1,0,2,1): 0, (1,0,2,2): 0,
#           (1,1,0,0): 0, (1,1,0,1): 0, (1,1,0,2): 0, (1,1,1,0): 0, (1,1,1,1): 0, (1,1,1,2): 0, (1,1,2,0): 0, (1,1,2,1): 0, (1,1,2,2): 0,
#           (1,2,0,0): 0, (1,2,0,1): 0, (1,2,0,2): 0, (1,2,1,0): 0, (1,2,1,1): 0, (1,2,1,2): 0, (1,2,2,0): 0, (1,2,2,1): 0, (1,2,2,2): 0,
#           (2,0,0,0): 0, (2,0,0,1): 0, (2,0,0,2): 0, (2,0,1,0): 0, (2,0,1,1): 0, (2,0,1,2): 0, (2,0,2,0): 0, (2,0,2,1): 0, (2,0,2,2): 0,
#           (2,1,0,0): 0, (2,1,0,1): 0, (2,1,0,2): 0, (2,1,1,0): 0, (2,1,1,1): 0, (2,1,1,2): 0, (2,1,2,0): 0, (2,1,2,1): 0, (2,1,2,2): 0,
#           (2,2,0,0): 0, (2,2,0,1): 0, (2,2,0,2): 0, (2,2,1,0): 0, (2,2,1,1): 0, (2,2,1,2): 0, (2,2,2,0): 0, (2,2,2,1): 0, (2,2,2,2): 0,
#          }



# política que atribui sempre a melhor trabalhadora para um serviço
MeanCost = env.Hxy/(env.Pwxy)

policy = {}
for s in env.states:
  if s[0] == 0:
    policy[s] = 0
  else:
    if min(MeanCost[:,s[0]-1]) < env.Gx[s[0]-1]:
      policy[s] = np.argmin(MeanCost[:,s[0]-1])+1
    else:
      policy[s] = 0


eval = SimulatePolicy(env, policy, maxSteps)

Steps: 100
(0, 0, 0, 0)
Congratulations. You earn 17.555765440764052.


Evaluate the previously defined policy with Monte Carlo simulation.

In [ ]:
maxSteps = 100
nSamples = 1000
e = EvaluatePolicyMC(env, policy, maxSteps, nSamples);

Simulating 1.0
The average gain in simulations was 14.91050737181926.


# Execution 3: Analytically Policy Evaluation

Expand the cells, before executing them.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def policy_evaluation(Environment,pi, gamma, epsilon):
    # Avaliação analítica.
    V = {}
    # Dicionário com todos os estados possíveis, inicializados em 0.
    for s in Environment.states:
        V[s] = 0

    res = float('inf')

    resolution = 10;

    steps = 0

    while res > epsilon:
        V_old = V.copy()
        res = 0

        # Para cada estado possível.
        for s in Environment.states:
            V[s] = 0
            # Encontra os próximos estados possíveis e suas probabilides.
            Transitions = Environment.nextState(s,pi[s])
            # Para cada um dos novos estados possíveis, atualiza a recomensa possível a partir do estado.
            for ss in Transitions:
                V[s] = V[s] + Transitions[ss]*(Environment.reward(s,pi[s],ss) + gamma*V_old[ss])

            if abs(V[s] - V_old[s]) > res:
                res = abs(V[s] - V_old[s])


        steps += 1
        if steps % resolution == 0:
            clear_output(wait=True)
            print(f'Residual {res}')

    return V[Environment.state], V

def drawValues(env,V):
    img = np.zeros((env.Y,env.X))
    for j in range(1,env.Y+1):
      for i in range(1,env.X+1):
        img[env.Y - j,i-1] = V[(i,j)]
    plt.imshow(img, cmap='gray', vmin=min(V.values()), vmax=max(V.values()))
    plt.show()

In [ ]:
gamma = 0.99
epsilon = 0.0001

# política que atribui sempre a melhor trabalhadora para um serviço
MeanCost = env.Hxy/(env.Pwxy)

policy = {}
for s in env.states:
  if s[0] == 0:
    policy[s] = 0
  else:
    if min(MeanCost[:,s[0]-1]) < env.Gx[s[0]-1]:
      policy[s] = np.argmin(MeanCost[:,s[0]-1])+1
    else:
      policy[s] = 0

eval, V = policy_evaluation(env,policy, gamma, epsilon)

# drawValues(env,V)
print(f'The expected cumulative reward is {eval}.')

Residual 9.960497842698146e-05
The expected cumulative reward is 15.425177870051483.


# Execution 4: Policy Iteration

Expand the cells before executing.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def policy_evaluation(Environment,pi, gamma, epsilon):
    # Dicionário com todos os estados possíveis, inicializados em 0.
    V = {}
    for s in Environment.states:
        V[s] = 0

    res = float('inf')

    while res > epsilon:
        V_old = V.copy()
        res = 0

        # Para cada estado.
        for s in Environment.states:
            V[s] = 0
            # Próximos estados possíveis a partir do estado analisado, e suas probabilidades.
            Transitions = Environment.nextState(s,pi[s])
            # Para cada um desses possíveis estados futuros, atualiza a recompensa.
            for ss in Transitions:
                V[s] = V[s] + Transitions[ss]*(Environment.reward(s,pi[s],ss) + gamma*V_old[ss])

            if abs(V[s] - V_old[s]) > res:
                res = abs(V[s] - V_old[s])

    return V

def policy_improvement(Environment,V, gamma):
    # Política.
    pi = {}
    # Para cada estado do ambiente.
    for s in Environment.states:
        # Dicionário para indicar a Função de Valor para cada ação possível, a partir do estado s.
        Q = {}
        # Para cada ação.
        for a in Environment.actions:
            Q[a] = 0
            # Próximos estados a partir do estado atual e da ação escolhida.
            Transitions = Environment.nextState(s,a)
            # Atualiza a Função de Valor.
            for ss in Transitions:
                Q[a] = Q[a] + Transitions[ss]*(Environment.reward(s,a,ss) + gamma*V[ss])

        # A política do ambiente para o estado s será tomar a ação que maximiza a Função de Valor.
        pi[s] = max(Q, key=Q.get)
    return pi

def policy_iteration(Environment, gamma, eps):

    res = 10

    # Cria uma política inicial (sempre indica "refuse").
    policy = {}
    for s in Environment.states:
      policy[s] = 0;

    # Avalia a política criada.
    V = policy_evaluation(Environment, policy, gamma, eps)


    # renderPolicy(Environment,policy)

    # input(f'O valor da política é {V[(1,1)]}. Pressione enter para continuar.')

    while res > eps:
        # Melhora a política.
        policy = policy_improvement(Environment,V, gamma)
        V_old = V.copy()
        # Avalia a nova política.
        V = policy_evaluation(Environment, policy, gamma, eps)

        res = np.max(  np.abs( np.array(list(V.values())) -  np.array(list(V_old.values())) ) )

        print(f'Residual {res}')

        # clear_output(wait=True)
        # renderPolicy(Environment,policy)

        print(f'O valor da política é {V[(0,0,0,0)]}.')

    return policy



In [ ]:
gamma = 0.99
eps = 0.0001

policy_iteration(env,gamma,eps);

Residual 1.3024656696020485
O valor da política é 1.1017324591747242.
Residual 0.0
O valor da política é 1.1017324591747242.


# Execution 5: Value Iteration

Expand the cells before executing.

In [ ]:
def value_iteration(Environment,gamma, eps):
    # Iteração de Valor, para encontrar uma política ótima.

    # Inicializa a função de valor, inicialmente com 0 para todos os estados.
    V = {}
    for s in Environment.states:
        V[s] = 0

    res = float('inf')

    # Definir epsilon (precisão desejada), conforme gamma (fator de desconto).                     ?
    if gamma < 1:
        epsilon = eps*(1-gamma)/(2*gamma)
    else:
        epsilon = eps

    while res > epsilon:
        V_old = V.copy()
        res = 0

#       Encontra a função valor epsilon-ótima
        # Para cada estado.
        for s in Environment.states:
            Q = {}
            # Testa a Função de Valor para todas as ações.
            for a in Environment.actions:
                Q[a] = 0
                Transitions = Environment.nextState(s,a)
                for ss in Transitions:
                    Q[a] = Q[a] + Transitions[ss]*(Environment.reward(s,a,ss) + gamma*V_old[ss])

            # A função valor do estado s é o maior valor obtido no teste das ações possíveis.
            V[s] = max(Q.values())

            if abs(V[s] - V_old[s]) > res:
                res = abs(V[s] - V_old[s])

        print(f'Residual {res}')

        clear_output(wait=True)

#   Extrai a política epsilon-ótima
    pi = {}
    for s in Environment.states:
        Q = {}
        for a in Environment.actions:
            Q[a] = 0
            Transitions = Environment.nextState(s,a)
            for ss in Transitions:
                Q[a] = Q[a] + Transitions[ss]*(Environment.reward(s,a,ss) + gamma*V[ss])

        pi[s] = max(Q, key=Q.get)


    return pi, V

In [ ]:
gamma = 0.99
epsilon = 0.01


policy, eval = value_iteration(env,gamma, epsilon)
# renderPolicy(env,policy)
print(f'O valor da política é {eval}.')

Residual 9.186989371512411e-05


In [ ]:
print('Ganho obtido com cada serviço')
print(env.Gx)

print('Custo médio de cada serviço para cada trabalhadora')
print(env.Hxy/(env.Pwxy))

print('Tempo médio de cada serviço para cada trabalhadora')
print(1/(env.Pwxy))


Ganho obtido com cada serviço
[1.20471269 1.27851004]
Custo médio de cada serviço para cada trabalhadora
[[0.36460998 0.84975446]
 [0.36307743 1.34929224]
 [6.57762171 0.46000512]]
Tempo médio de cada serviço para cada trabalhadora
[[1.21760547 1.39190045]
 [2.22895737 2.051881  ]
 [7.96152948 1.10675432]]


In [ ]:
policy

{(0, 0, 0, 0): 0,
 (0, 0, 0, 1): 0,
 (0, 0, 0, 2): 0,
 (0, 0, 1, 0): 0,
 (0, 0, 1, 1): 0,
 (0, 0, 1, 2): 0,
 (0, 0, 2, 0): 0,
 (0, 0, 2, 1): 0,
 (0, 0, 2, 2): 0,
 (0, 1, 0, 0): 0,
 (0, 1, 0, 1): 0,
 (0, 1, 0, 2): 0,
 (0, 1, 1, 0): 0,
 (0, 1, 1, 1): 0,
 (0, 1, 1, 2): 0,
 (0, 1, 2, 0): 0,
 (0, 1, 2, 1): 0,
 (0, 1, 2, 2): 0,
 (0, 2, 0, 0): 0,
 (0, 2, 0, 1): 0,
 (0, 2, 0, 2): 0,
 (0, 2, 1, 0): 0,
 (0, 2, 1, 1): 0,
 (0, 2, 1, 2): 0,
 (0, 2, 2, 0): 0,
 (0, 2, 2, 1): 0,
 (0, 2, 2, 2): 0,
 (1, 0, 0, 0): 1,
 (1, 0, 0, 1): 2,
 (1, 0, 0, 2): 1,
 (1, 0, 1, 0): 1,
 (1, 0, 1, 1): 1,
 (1, 0, 1, 2): 1,
 (1, 0, 2, 0): 1,
 (1, 0, 2, 1): 1,
 (1, 0, 2, 2): 1,
 (1, 1, 0, 0): 2,
 (1, 1, 0, 1): 2,
 (1, 1, 0, 2): 2,
 (1, 1, 1, 0): 0,
 (1, 1, 1, 1): 0,
 (1, 1, 1, 2): 0,
 (1, 1, 2, 0): 0,
 (1, 1, 2, 1): 0,
 (1, 1, 2, 2): 0,
 (1, 2, 0, 0): 2,
 (1, 2, 0, 1): 2,
 (1, 2, 0, 2): 2,
 (1, 2, 1, 0): 0,
 (1, 2, 1, 1): 0,
 (1, 2, 1, 2): 0,
 (1, 2, 2, 0): 0,
 (1, 2, 2, 1): 0,
 (1, 2, 2, 2): 0,
 (2, 0, 0, 0): 3,
 (2, 0, 0,